# SQL Queries with Minio (& PowerBI)

Minio implements the [S3 SELECT API](https://docs.min.io/docs/minio-select-api-quickstart-guide.html). It is not effective for creating joins or other relational database tricks, but it's phenomenal at extracting exactly the data that you need, so that your queries are blazingly fast. 


For reference on how to use this SQL flavour, look at

[The AWS reference](https://docs.aws.amazon.com/AmazonS3/latest/dev/s3-glacier-select-sql-reference-select.html)


*Note: Amazon S3 Select does not support whole-object compression for Parquet objects.*
[Source](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.select_object_content)

**NOTE: The examples here use JSON, but CSV is better suited to large datasets, performing 10x faster in my experiment.**

**NOTE: The .Parquet example is commented out. Parquet is DISABLED by default since hostile crafted input can easily crash the server.**
[Source](https://docs.min.io/docs/minio-select-api-quickstart-guide.html)

In [1]:
!pip install pyarrow

     |████████████████████████████████| 21.9 MB 124 kB/s  eta 0:00:01       | 4.3 MB 40.8 MB/s eta 0:00:01     |███████████▉                    | 8.1 MB 40.8 MB/s eta 0:00:01     |█████████████████               | 11.7 MB 68.6 MB/s eta 0:00:01


## Connect to storage

In [2]:
import daaas_storage_boto3 as storage

s3 = storage.get_standard_client()

BUCKET = "shared"

# Fast SQL Extractions and pandas (can be used with PowerBI)

Minio implements the S3 Select API, which reads a minimal amount of data off of disk. This makes the queries very fast, even on large tables. Also, you can read the data straight out of a file, without creating or managing a complex database.

**PowerBI**: You can use these snippets to load pandas dataframes into PowerBI. Check out [the PowerBI tutorial](https://docs.microsoft.com/en-us/power-bi/connect-data/desktop-python-scripts). **note:** this only works with pandas, not arrow. So use `storage.pandas_from_json`. **Do not use** `storage.arrow_from_json`.

## Query your data with SQL (.csv.gz)

In [3]:
%%time

r = s3.select_object_content(
    Bucket=BUCKET,
    Key='/blair-drummond/sql-example/TotalPopulation.csv.gz',
    ExpressionType='SQL',
    # Note, there's no ';' at the end.
    Expression="""
    SELECT PopTotal,PopDensity FROM s3object s 
    WHERE s.Location like '%Canada%'
    """,
    InputSerialization={
        'CSV': {
            # Use this if your CSV file has a header. Else set to "NONE".
            "FileHeaderInfo": "USE",
            'RecordDelimiter': '\n',
            'FieldDelimiter': ',',
        },
        # Remove this if the file isn't compressed.
        'CompressionType': 'GZIP',
    },
    OutputSerialization={'JSON': {}},
    #OutputSerialization={'CSV': {'RecordDelimiter': '\n', 'FieldDelimiter': ','}},
)

df = storage.pandas_from_json(r)
#df = storage.pandas_from_csv(r)
df.head()

CPU times: user 20.6 ms, sys: 12.9 ms, total: 33.4 ms
Wall time: 1.39 s


Created Dataframe with dimensions: (nrow, ncol) = (884, 2)


,PopTotal,PopDensity
0,13733.398,1.510
1,14078.449,1.548
2,14445.453,1.589
3,14834.905,1.631
4,15245.416,1.677


## Query your data with SQL (.parquet)

**NOTE: If you're running this on PowerBI, you'll need either pyarrow or fastparquet installed.** 

**Note:** You should not compress your parquet files[^1]!!! They can be larger compressed, and the S3 Select API does not support querying them.

**NOTE: The .Parquet example is commented out. Parquet is DISABLED by default since hostile crafted input can easily crash the server.**
[Source](https://docs.min.io/docs/minio-select-api-quickstart-guide.html)

[^1]: Unless you use SNAPPY. But BZIP2 and GZIP are not supported.

In [4]:
## https://docs.min.io/docs/minio-select-api-quickstart-guide.html 

#Enabling Parquet Format

#Parquet is DISABLED by default since hostile crafted input can easily crash the server.
# %%time

# r = s3.select_object_content(
#     Bucket=BUCKET,
#     Key='/blair-drummond/sql-example/TotalPopulation.parquet',
#     ExpressionType='SQL',
#     Expression="SELECT * FROM s3object s WHERE s.Location like '%Canada%'",
#     InputSerialization={
#         'Parquet': {},
#         'CompressionType': 'NONE',
#     },
#     OutputSerialization={'JSON': {}},
# )

# df = storage.pandas_from_json(r)
# df.head()

## Query your data with SQL (.csv)

Note, you'll probably get **significant** storage savings if you compress your csv files. (Read: 10gb -> 500mb, for example). So if it's under your control, it's recommended to use gzip. The S3 Select API that we're using also has some support for BZIP2. *(You can also use SNAPPY on `.parquet` files)*

[S3 Select Compression Support](https://aws.amazon.com/about-aws/whats-new/2018/09/amazon-s3-announces-new-features-for-s3-select/)

In [5]:
%%time

r = s3.select_object_content(
    Bucket=BUCKET,
    Key='/blair-drummond/sql-example/TotalPopulation.csv',
    ExpressionType='SQL',
    # Note, there's no ';' at the end.
    Expression="""
    SELECT PopTotal,PopDensity FROM s3object s 
    WHERE s.Location like '%Canada%'
    """,
    InputSerialization={
        'CSV': {
            # Use this if your CSV file has a header. Else set to "NONE".
            "FileHeaderInfo": "USE",
        },
        # Remove this if the file isn't compressed.
        # 'CompressionType': 'GZIP',
    },
    # JSON is easier to work with than csv, unless you
    # have a massive amount of data.
    OutputSerialization={'JSON': {}},
)

df = storage.pandas_from_json(r)
df.head()

CPU times: user 28.2 ms, sys: 4.14 ms, total: 32.3 ms
Wall time: 1.68 s


Created Dataframe with dimensions: (nrow, ncol) = (884, 2)


,PopTotal,PopDensity
0,13733.398,1.510
1,14078.449,1.548
2,14445.453,1.589
3,14834.905,1.631
4,15245.416,1.677


## NOTE: Json v.s. CSV

JSON transmits more data than CSV, so **if performance is key, use csv**.

**The disadvantage of CSV, is that the S3 API for CSV doesn't return you column names.**

However, you can run a small JSON query, then manually stitch together the column names.

Compare the times below.

In [6]:
%%time

r = s3.select_object_content(
    Bucket=BUCKET,
    Key='/blair-drummond/sql-example/TotalPopulation.csv.gz',
    ExpressionType='SQL',
    # Note, there's no ';' at the end.
    Expression="""
    SELECT PopTotal,PopDensity FROM s3object s 
    """,
    InputSerialization={
        'CSV': {
            # Use this if your CSV file has a header. Else set to "NONE".
            "FileHeaderInfo": "USE",
            'RecordDelimiter': '\n',
            'FieldDelimiter': ',',
        },
        # Remove this if the file isn't compressed.
        'CompressionType': 'GZIP',
    },
    OutputSerialization={'JSON': {}},
    #OutputSerialization={'CSV': {'RecordDelimiter': '\n', 'FieldDelimiter': ','}},
)

df = storage.pandas_from_json(r)
#df = storage.pandas_from_csv(r)
df.head()

CPU times: user 895 ms, sys: 150 ms, total: 1.04 s
Wall time: 5.99 s


Created Dataframe with dimensions: (nrow, ncol) = (280932, 2)


,PopTotal,PopDensity
0,7752.117,11.874
1,7840.151,12.009
2,7935.996,12.156
3,8039.684,12.315
4,8151.316,12.486


In [7]:
%%time

r = s3.select_object_content(
    Bucket=BUCKET,
    Key='/blair-drummond/sql-example/TotalPopulation.csv.gz',
    ExpressionType='SQL',
    # Note, there's no ';' at the end.
    Expression="""
    SELECT PopTotal,PopDensity FROM s3object s 
    """,
    InputSerialization={
        'CSV': {
            # Use this if your CSV file has a header. Else set to "NONE".
            "FileHeaderInfo": "USE",
            'RecordDelimiter': '\n',
            'FieldDelimiter': ',',
        },
        # Remove this if the file isn't compressed.
        'CompressionType': 'GZIP',
    },
    #OutputSerialization={'JSON': {}},
    OutputSerialization={'CSV': {'RecordDelimiter': '\n', 'FieldDelimiter': ','}},
)

#df = storage.pandas_from_json(r)
df = storage.pandas_from_csv(r)
df.head()

CPU times: user 87.1 ms, sys: 15.1 ms, total: 102 ms
Wall time: 1.5 s


Created Dataframe with dimensions: (nrow, ncol) = (280932, 2)


,0,1
0,7752.117,11.874
1,7840.151,12.009
2,7935.996,12.156
3,8039.684,12.315
4,8151.316,12.486


# Beyond Pandas: Arrow v.s. Pandas, CSV v.s. Parquet

*Apache Arrow* is a newer tool designed for efficient data storage and retrieval. It's how Pandas opens `.parquet` files. We're going to do some benchmarking here, and we'll look at an experiment with the following variables.

1. File Format
    - `.csv` 
    - `.csv.gz` 
    - `.parquet`
    
2. Query type
    - *Row Extraction*
    - *Column Extraction*
    
3. Python Tool
    - `pyarrow`
    - `pandas`


We're going to test every combination of these, to see how they work with eachother.

In [8]:
################################################
###           Arrow versus Pandas            ### 
################################################
import time
def timing(f):
    """ Discard the output of the function, but get the time. """
    def wrap(*args):
        time1 = time.time()
        f(*args)
        time2 = time.time()
        # milliseconds
        ms = (time2-time1)*1000.0
        return ms
    return wrap

funcs = {
    'arrow'  : timing(storage.arrow_from_json),    
    'pandas' : timing(storage.pandas_from_json)
}

In [9]:
################################################
###       Row versus Column Operations       ###
################################################

## Query is adjusted so that roughly 
## the same amount of data is scanned.
sql = { 
        # nrow = 4420, ncol = 2;   nrow*ncol = 8840
        'column' : """
        SELECT PopTotal,PopDensity FROM s3object s 
        LIMIT 4420
        """,
        # nrow = 884, ncol = 10;  nrow*ncol = 8840
        'row' : """
        SELECT * FROM s3object s 
        WHERE s.Location like '%Canada%'
        """
}

In [10]:
################################################
###                File Format               ###
################################################

## Note that csv.gz is smaller than parquet!

def exp_csv(sql_query):
    return s3.select_object_content(
        Bucket=BUCKET,
        # File size = 21 mb
        Key='/blair-drummond/sql-example/TotalPopulation.csv',
        ExpressionType='SQL',
        Expression=sql_query,
        InputSerialization={'CSV': {"FileHeaderInfo": "USE"}},
        OutputSerialization={'JSON': {}}
    )


def exp_csv_gz(sql_query):
    return s3.select_object_content(
        Bucket=BUCKET,
        # File size = 5.6 mb
        Key='/blair-drummond/sql-example/TotalPopulation.csv.gz',
        ExpressionType='SQL',
        Expression=sql_query,
        InputSerialization={
            'CSV': {"FileHeaderInfo": "USE"},
            'CompressionType': 'GZIP',
        },
        OutputSerialization={'JSON': {}}
    )


def exp_parquet(sql_query):
    return s3.select_object_content(
        Bucket=BUCKET,
        # File size = 6.8 mb
        Key='/blair-drummond/sql-example/TotalPopulation.parquet',
        ExpressionType='SQL',
        Expression=sql_query,
        InputSerialization={'Parquet': {}},
        OutputSerialization={'JSON': {}}
    )

formats = {
    'csv'     : exp_csv,
    'csv.gz'  : exp_csv_gz,
    'parquet' : exp_parquet   
}

## Run the experiment!

**Note: ** 
* Parquet is dsabled by default since hostile crafted input can easily crash the server
* To enable Parquet set the environment variable MINIO_API_SELECT_PARQUET=on

In [11]:
import pandas as pd

In [12]:
### By Column

col_exp = lambda backend,file: funcs[backend](formats[file](sql['column']))

cols = pd.DataFrame({
    'csv'     : [ col_exp('pandas', 'csv'),     col_exp('arrow', 'csv')      ],
    'csv.gz'  : [ col_exp('pandas', 'csv.gz'),  col_exp('arrow', 'csv.gz')   ]
    # 'parquet' : [ col_exp('pandas', 'parquet'), col_exp('arrow', 'parquet')  ]
    }, index=['pandas', 'arrow'])

cols

Created Dataframe with dimensions: (nrow, ncol) = (4420, 2)
Created Dataframe with dimensions: (nrow, ncol) = (4420, 2)
Created Dataframe with dimensions: (nrow, ncol) = (4420, 2)
Created Dataframe with dimensions: (nrow, ncol) = (4420, 2)


,csv,csv.gz
pandas,139.540672,106.382608
arrow,53.184986,44.803619


In [13]:
### By Row

row_exp = lambda backend,file: funcs[backend](formats[file](sql['row']))

rows = pd.DataFrame({
    'csv'     : [ row_exp('pandas', 'csv'),     row_exp('arrow', 'csv')      ],
    'csv.gz'  : [ row_exp('pandas', 'csv.gz'),  row_exp('arrow', 'csv.gz')   ]
    # 'parquet' : [ row_exp('pandas', 'parquet'), row_exp('arrow', 'parquet')  ]
    }, index=['pandas', 'arrow'])

rows

Created Dataframe with dimensions: (nrow, ncol) = (884, 10)
Created Dataframe with dimensions: (nrow, ncol) = (884, 10)
Created Dataframe with dimensions: (nrow, ncol) = (884, 10)
Created Dataframe with dimensions: (nrow, ncol) = (884, 10)


,csv,csv.gz
pandas,674.450397,581.545830
arrow,657.702208,690.969706


**NOTE: I think Parquet will probably perform much better as the file size increases. Our files here are pretty small.** 

## Conclusion: arrow > pandas.

This experiment done with a very small dataset, but there are two observations:

1. **Arrow is faster than pandas in every case**.
2. **Scanning columns is WAY faster than scanning Rows**.

Also, note that while `csv.gz` is slightly slower than `csv`, the `csv.gz` files are `1/4` the size in storage. For large files, this will translate to faster transfer speeds.